<img src="https://teaching.bowyer.ai/sdsai/resources/0/img/IMPERIAL_logo_RGB_Blue_2024.svg" alt="Imperial Logo" width="500"/><br /><br />

ML Foundations and Data Preparation - Tutorial Exercise Solutions
==============
### SURG70098 - Surgical Data Science and AI
### Stuart Bowyer

# Setup

In [ ]:
# Install and import
%pip install pandas
%pip install matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import pandas_gbq

# Exercise 3.1


In [ ]:
# @markdown Enter your Google Cloud Project ID:
project_id = 'mimic-project-345345'  # @param {type:"string"}

df_day1_vitalsign = pandas_gbq.read_gbq("""
 SELECT *
 FROM `physionet-data.mimiciv_derived.first_day_vitalsign`
 LEFT JOIN (
 SELECT
 subject_id,
 stay_id,
 gender,
 race,
 admission_age,
 dod IS NOT NULL AS mortality
 FROM
 `physionet-data.mimiciv_derived.icustay_detail`
 )
 USING(subject_id, stay_id)
 WHERE heart_rate_mean IS NOT NULL
 LIMIT 10000
""", project_id=project_id)

## What are the associations between different continuous variables and mortality?

In [ ]:
# I will create a function to make these plots easier for lots of columns
def plot_vs_mortality(column):

  fig, axes = plt.subplots(ncols=3,figsize=(12,4))
  df_day1_vitalsign.boxplot(column=column, by='mortality', ax = axes[0])
  df_day1_vitalsign[df_day1_vitalsign.mortality == True][column].hist(label='Mortality True', ax = axes[1])
  df_day1_vitalsign[df_day1_vitalsign.mortality == False][column].hist(label='Mortality False', ax = axes[1])
  axes[1].legend()
  df_day1_vitalsign[df_day1_vitalsign.mortality == True][column].plot.kde(label='Mortality True', ax = axes[2])
  df_day1_vitalsign[df_day1_vitalsign.mortality == False][column].plot.kde(label='Mortality False', ax = axes[2])
  plt.show()


plot_vs_mortality("heart_rate_mean")
plot_vs_mortality("spo2_mean")
plot_vs_mortality("sbp_mean")
plot_vs_mortality("dbp_mean")

## Which continuous variables are most/least correlated with admission_age?

In [ ]:
mean_value_columns = df_day1_vitalsign.columns[df_day1_vitalsign.columns.str.match('.*_mean|.*_age')].tolist()

df_day1_vitalsign[mean_value_columns].corr()['admission_age']



## Which continuous variables have the highest and lowest variation?

In [ ]:
df_day1_vitalsign.describe().loc['std'].sort_values(ascending=False)

# Could also calculate IQR, or Range

# Exercise 3.4

In [ ]:
df_messy = pd.read_csv("https://teaching.bowyer.io/SDSAI/3/data/messy_data.csv")
df_clean = df_messy

# Clean outliers from ages (could possibly have used clip)
df_clean['Age'] = df_clean['Age'].mask((df_clean['Age'] < 0) | (df_clean['Age'] > 120))

# Clean weight
df_clean[['WeightKg','WeightUnit']] = df_clean['Weight'].str.extract(r'([\d.]+)\s*([a-zA-Z]*)')
df_clean['WeightKg'] = df_clean['WeightKg'].mask(df_clean['WeightUnit'] == '').astype(float)
df_clean['WeightKg'] = df_clean['WeightKg'].where(
    df_clean['WeightUnit'].str.startswith('kg'),
    df_clean['WeightKg'] / 2.205
)
df_clean.drop(columns=['Weight', 'WeightUnit'], inplace=True)

# Clean height
# This one is quite complex, but possible with a few carefull conditions

# Clean blood pressure
df_clean[['SystolicBP','DiastolicBP']] = df_clean['BloodPressure'] \
    .str.split('/', expand=True) \
    .astype('Int64')
df_clean.drop(columns=['BloodPressure'], inplace=True)

# Clean temperature
df_clean[['TempC','TempUnit']] = df_clean['Temp'].str.extract(r'([\d.]+)\s*([CF]*)')
df_clean['TempC'] = df_clean['TempC'].mask(df_clean['TempUnit'] == '').astype(float)
df_clean['TempC'] = df_clean['TempC'].where(
    df_clean['TempUnit'] == 'C',
    (df_clean['TempC'] - 32) * (5 / 9)
)
df_clean.drop(columns=['Temp','TempUnit'], inplace=True)

# Clean gender
df_clean['Gender'] = df_clean['Gender'].str.upper().str[0]

# Clean blood glucose clean
df_clean['BGMethod'] = 'BG' + df_clean['BGMethod']
df_clean = pd.merge(df_clean, df_clean.pivot(columns='BGMethod', index='PatientID', values='BloodGlucose'), on='PatientID')
df_clean.drop(columns=['BloodGlucose', 'BGMethod'], inplace=True)

# Clean appointment Date
df_clean['AppointmentDate'] = pd.to_datetime(df_clean['AppointmentDate'], format='mixed', errors='coerce')

# Impute missing values
df_clean.fillna({
    'Age': df_clean['Age'].mean(),
    'WeightKg': df_clean['WeightKg'].mean(),
    'SystolicBP': round(df_clean['SystolicBP'].mean()),
    'DiastolicBP': round(df_clean['DiastolicBP'].mean()),
    'TempC': df_clean['TempC'].mean(),
    'Diagnosis': 'None',
}, inplace=True)

df_clean

# Exercise 3.3

In [ ]:
import pandas as pd
import pandas_gbq

project_id = 'mimic-project-439314'  # <--- INPUT YOUR PROJECT ID HERE

In [ ]:
df_admissions = pandas_gbq.read_gbq("""
  SELECT *
  FROM `physionet-data.mimiciv_hosp.admissions`
  WHERE MOD(subject_id, 10) = 0
""", project_id=project_id)

In [ ]:
df_age = pandas_gbq.read_gbq("""
  SELECT subject_id, hadm_id, age
  FROM `physionet-data.mimiciv_derived.age`
  WHERE MOD(subject_id, 10) = 0
""", project_id=project_id)

In [ ]:
df_diagnoses_icd = pandas_gbq.read_gbq("""
  SELECT subject_id, hadm_id, icd_code, icd_version
  FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
  WHERE MOD(subject_id, 10) = 0
""", project_id=project_id)

In [ ]:
df_d_icd_diagnoses = pandas_gbq.read_gbq("""
  SELECT *
  FROM `physionet-data.mimiciv_hosp.d_icd_diagnoses`
""", project_id=project_id)

In [ ]:
df_procedures_icd = pandas_gbq.read_gbq("""
  SELECT subject_id, hadm_id, icd_code, icd_version
  FROM `physionet-data.mimiciv_hosp.procedures_icd`
  WHERE MOD(subject_id, 10) = 0
""", project_id=project_id)

In [ ]:
df_d_icd_procedures = pandas_gbq.read_gbq("""
  SELECT *
  FROM `physionet-data.mimiciv_hosp.d_icd_procedures`
""", project_id=project_id)

## Preliminary EDA
Some preliminary analysis to see how many cesarean section procedures and diagnoses of interest we have.

In [ ]:
proc_count = df_procedures_icd \
  .groupby(['icd_code','icd_version']) \
  .count() \
  .sort_values('hadm_id', ascending=False) \
  .merge(df_d_icd_procedures, on='icd_code')

proc_count[proc_count.icd_code.str.contains('^74[0-9]$')]

In [ ]:
diag_count = df_diagnoses_icd \
  .groupby(['icd_code','icd_version']) \
  .count() \
  .sort_values('hadm_id', ascending=False) \
  .merge(df_d_icd_diagnoses, on='icd_code')

# Find all the diabetes diagnoses
diag_count[diag_count.icd_code.str.contains('^250[0-9].$')].head()

## Get the list of admissions for the cohort of interest
i.e. the admissions where there was a Cesarean section ICD-9 procedure code from 740 - 749

In [ ]:
hadm_list = df_procedures_icd[df_procedures_icd.icd_code.str.contains('^74[0-9]$')]['hadm_id']

## EDA Admission Data Frame

In [ ]:
# Separate out just the admission that contained a Cesarean section
df_cesa_admissions = df_admissions[df_admissions.hadm_id.isin(hadm_list)].copy()

# Engineer a new feature, called hadm_duration (and one for the duration in days)
df_cesa_admissions['hadm_duration'] = df_cesa_admissions['dischtime'] - df_cesa_admissions['admittime']
df_cesa_admissions['hadm_duration_days'] = df_cesa_admissions['hadm_duration'].dt.total_seconds() / 86400

# Engineer our output label feature
df_cesa_admissions['hadm_long'] = df_cesa_admissions['hadm_duration_days'] > 7

In [ ]:
# EDA Long Admissions
df_cesa_admissions['hadm_long'].value_counts()

In [ ]:
df_cesa_admissions['hadm_long'].value_counts(normalize=True)

In [ ]:
# EDA Admission Type
df_cesa_admissions.admission_type.value_counts()

In [ ]:
# EDA Admission Type Grouped By Long Admission or Not
df_cesa_admissions.groupby('hadm_long').admission_type.value_counts(normalize=True)

This EDA potentially substantiates the use of admission_type as a feature, as the rate of emergency admissions is higher for the long admission group

In [ ]:
# EDA race
df_cesa_admissions.race.value_counts()

## EDA Age Data Frame

In [ ]:
# Separate out just the ages for admission that contained a Cesarean section
df_cesa_age = df_age[df_age.hadm_id.isin(hadm_list)].copy()


# Plot ages
df_cesa_age.age.hist(bins=7)

## EDA Diagnosis Data Frame

In [ ]:
# Separate out just the diagnoses for admission that contained a Cesarean section
df_cesa_diagnoses = df_diagnoses_icd[df_diagnoses_icd.hadm_id.isin(hadm_list)].copy()

# Find the diagnoses for diabetes 250.00-99
df_cesa_diagnoses['diag_diabetes'] = df_cesa_diagnoses.icd_code.str.contains('^250')

# Drop all the false rows and non-used columns
# This is just to keep the data as small as possible to improve efficiency
df_cesa_diagnoses = df_cesa_diagnoses[df_cesa_diagnoses['diag_diabetes']] \
  .drop(['subject_id', 'icd_code', 'icd_version'], axis=1)

df_cesa_diagnoses

## Join Data Frames to Create Feature Table

In [ ]:
df_features = df_cesa_admissions[['hadm_long', 'hadm_id', 'admission_type', 'race']] \
  .merge(df_cesa_age[['hadm_id', 'age']], on='hadm_id') \
  .merge(df_cesa_diagnoses, how='left', on='hadm_id') \
  .fillna({'diag_diabetes': False}, inplace = False)

df_features

## EDA of Merged Feature Table
Some of the features needed to be merged for EDA

In [ ]:
df_features.groupby('hadm_long').age.describe()

This potentially substantitates that age is a useful feature as the mean/median/max for the long admissions are higher than the others

In [ ]:
df_features.groupby('hadm_long').diag_diabetes.value_counts(normalize=True)

Again, potentially substantiates that diabetes is a useful feature as there is a higher rate in the long admission cohort

## Encode the Features


### `admission_type`
I made the choice to agregate direct and EW (emergency ward) admissions and then use an ordinal encoding where:

1.  'OBSERVATION ADMIT'
1.  'URGENT'
1.  'DIRECT EMER.' / 'EW EMER.'


In [ ]:
ordinal_mapping = {
    'OBSERVATION ADMIT': 0,
    'URGENT': 1,
    'DIRECT EMER.': 2,
    'EW EMER.': 2
}

df_features['admission_type'] = df_features['admission_type'].map(ordinal_mapping)

### `race`

First, I aggregated the race categories to ensure we have sufficient samples per category. This approach is an example, in practice, you should be careful aggregating race/ethnicity as there are many implications and sensitivities. I suggest using something like the NHS categorisation to do this, if possible. Realistically, this is still probably too many categories and would need further aggregation.

Second, I used one-hot encoding for race as there is no clear way of ordinal encoding this data.

In [ ]:
# This aggregates the 'race' column in three ways:
#   1. Drops any subclassifications with a "- SUBCLASSIFICATION"
#   2. Swaps an "OR" to a "/" to combine two identical groups
#   3. Swaps various "UNKNOWN/UNDEFINED" categories together
df_features['race'] = df_features['race'] \
  .str.replace(r" -.*$", "", regex=True) \
  .str.replace(" OR ", "/") \
  .str.replace(r"^(UNKNOWN|UNABLE TO OBTAIN|PATIENT DECLINED TO ANSWER|OTHER)$", 'UNKNOWN/UNDEFINED', regex=True)

df_features = pd.get_dummies(df_features, columns=['race'])

## Polishing
This last bit of code polishes the feature table by removing the unneeded columns (hadm_id)

In [ ]:
df_features.drop(['hadm_id'], axis=1, inplace=True)

df_features